# Topic modelling of news headlines for prediction of news category with 7 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import re

import nlp_preprocessors

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Clean text

In [7]:
train['title_cleaned'] = train['title'].apply(lambda x: nlp_preprocessors.clean_text(x))

In [8]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt


## Extract features from 'title' 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [11]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [12]:
dtm

<67513x21412 sparse matrix of type '<class 'numpy.int64'>'
	with 330201 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [15]:
len(cv.get_feature_names_out())

21412

In [16]:
cv.get_feature_names_out()

array(['000', '034', '039', ..., 'войны', 'российские', 'украинцы'],
      dtype=object)

In [17]:
len(lda.components_)

7

In [18]:
lda.components_

array([[ 0.14348793, 18.51293758,  1.26928899, ...,  0.14285737,
         0.14285737,  0.14285766],
       [ 1.4931218 ,  7.11578264,  0.18207471, ...,  2.14285544,
         2.14285544,  0.14285779],
       [12.86028139, 13.05717921, 27.06406766, ...,  0.14285742,
         0.14285742,  0.14285776],
       ...,
       [ 0.14291442, 12.40353645, 22.27189586, ...,  0.14285744,
         0.14285744,  0.14285782],
       [ 0.14320841,  5.59882667,  0.14294238, ...,  0.14285742,
         0.14285742,  0.14285777],
       [ 2.07410323, 10.8853864 ,  0.14289863, ...,  0.14285745,
         0.14285745,  2.14285337]])

In [19]:
len(lda.components_[0])

21412

### Show most important words of first extracted topic

In [20]:
first_topic = lda.components_[0]

In [21]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([20969, 15342, 16516, ..., 15091, 10150, 18325], dtype=int64)

In [22]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [23]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

liveblog
selenskyj
angriff
kiew
lage
putin
russisch
russland
krieg
ukraine


### Show most important words of first extracted topic

In [24]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['stimme', 'invasion', 'moskau', 'ukrainisch', 'nato', 'liveblog', 'selenskyj', 'angriff', 'kiew', 'lage', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['johnson', 'großbritannien', 'eu', 'fdp', 'spd', 'land', 'panzer', 'faeser', 'streit', 'wahrheit', 'habeck', 'lindner', 'regierung', 'ampel', 'euro']


Die TOP-15 Wörter für das Thema #2
['usa', 'biden', 'us', 'million', 'fußball', 'gut', 'welt', 'taiwan', 'groß', 'analyse', 'berlin', 'wm', 'deutschland', 'scholz', 'china']


Die TOP-15 Wörter für das Thema #3
['podcast', 'nordkorea', 'tod', 'russland', 'haft', 'bahn', 'putin', 'ex', 'wahl', 'klima', 'partei', 'türkei', 'israel', 'afd', 'deutsch']


Die TOP-15 Wörter für das Thema #4
['mindestens', 'retten', 'europa', 'merz', 'steingart', 'gabor', 'spd', 'sanktion', 'gas', 'gastbeitrag', 'frau', 'grüne', 'cdu', 'tote', 'eu']


Die TOP-15 Wörter für das Thema #5
['stark', 'mensch', 'präsident', 'news', 'us',

### Get topics with highest probability for news items in train data

In [25]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [26]:
topic_results[0].round(5)

array([0.01787, 0.40656, 0.01789, 0.25146, 0.1279 , 0.16044, 0.01787])

In [27]:
topic_results[0].argmax()

1

In [28]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [29]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,1
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,1
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,1


## Visualize model

In [30]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0     -13.997216  36.421318       1        1  17.318913
2      17.531588 -15.635834       2        1  14.534681
5      32.238777  10.958028       3        1  14.226685
1     -28.602219   9.705539       4        1  14.116559
4      16.454649  36.964485       5        1  13.444289
6       1.808764  10.358519       6        1  13.190137
3     -12.819125 -16.301262       7        1  13.168736, topic_info=              Term         Freq        Total Category  logprob  loglift
18325      ukraine  4527.000000  4527.000000  Default  30.0000  30.0000
10150        krieg  2623.000000  2623.000000  Default  29.0000  29.0000
15087     russisch  1378.000000  1378.000000  Default  28.0000  28.0000
3583        corona  1140.000000  1140.000000  Default  27.0000  27.0000
15091     russland  1759.000000  1759.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
15666       scholz   192.173462  1012.939430   Topic7  -5.4284   0.3651
9895      konflikt   131.267303   371.197752   Topic7  -5.8096   0.9878
7467          groß   153.611149   764.239396   Topic7  -5.6524   0.4229
3899   deutschland   167.144920  1666.557649   Topic7  -5.5680  -0.2723
7716        halten   124.412448   316.394248   Topic7  -5.8632   1.0939

[433 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
8          7  0.984050       11km
63         5  0.992497         49
138        2  0.981847        abe
301        7  0.973207    absolut
418        2  0.153321        afd
...      ...       ...        ...
21131      3  0.121178    zweiter
21131      5  0.010537    zweiter
21131      6  0.231819    zweiter
21131      7  0.637501    zweiter
21275      1  0.995447  überblick

[844 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 5, 7, 4])

## Save train set with topics and dump model

In [31]:
#Save result to csv
train.to_csv('train_lda_topics_7.csv')

In [32]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv.jl')

['models/cv.jl']

In [33]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_7_topics.jl')

['models/lda_model_7_topics.jl']